In [0]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from sklearn.model_selection import train_test_split
import time
from keras import optimizers

Using TensorFlow backend.


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

In [0]:
import os
os.chdir('drive')

In [0]:
#data_train = pd.read_csv('v1/Preprocessed_using_translator_train.csv')
data_train = pd.read_csv('Preprocessed_using_translator_train.csv')
data_train.shape

(400000, 3)

In [0]:
#data_test= pd.read_csv("v1/Preprocessed_using_translator_test.csv")
data_test= pd.read_csv("Preprocessed_using_translator_test.csv")
data_test.shape

(100000, 2)

In [0]:
tokenize = Tokenizer(num_words= 10)
tokenize.fit_on_texts(['Hello, today is a good day hello Hello hi!'])
sequencd = tokenize.texts_to_sequences(['Hello, is today a good!'])
print(sequencd)
data = pad_sequences(sequencd)
print(data)

print(tokenize)

[[1, 3, 2, 4, 5]]
[[1 3 2 4 5]]


In [0]:
data_train["text"] = data_train["text"].apply(str)
data_test["text"] = data_test["text"].apply(str)
ylabels = pd.get_dummies(data_train.iloc[:,2].values)
vocabulary_size = 10000 
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_train["text"].values)
sequences = tokenizer.texts_to_sequences(data_train["text"].values)
data = pad_sequences(sequences)


In [0]:
xtrain, xval, ytrain, yval = train_test_split(data, ylabels, test_size=0.10, random_state=960)
print(xtrain.shape, xval.shape, ytrain.shape, yval.shape)

(360000, 988) (40000, 988) (360000, 3) (40000, 3)


In [0]:
sequence_length = data.shape[1]
embedding_dim = 76
num_filters = 32
filter_sizes = [33, 33, 33]
drop = 0.2

inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

c0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

DP0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(c0)
DP1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(c1)
DP2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(c2)

CT = Concatenate(axis=1)([DP0, DP1, DP2])

flatten = Flatten()(CT)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax')(dropout)

batch_size=32
epochs = 10

model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

st = time.time()
model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(xval, yval))  # starts training
en = time.time()

Train on 360000 samples, validate on 40000 samples
Epoch 1/10
 21312/360000 [>.............................] - ETA: 14:26 - loss: 0.5253 - acc: 0.7532

359968/360000 [============================>.] - ETA: 0s - loss: 0.2445 - acc: 0.8962

360000/360000 [==============================] - 922s 3ms/step - loss: 0.2445 - acc: 0.8962 - val_loss: 0.1816 - val_acc: 0.9258

Epoch 00001: val_acc improved from -inf to 0.92578, saving model to weights.001-0.9258.hdf5
Epoch 2/10
  7776/360000 [..............................] - ETA: 14:32 - loss: 0.1807 - acc: 0.9274

359968/360000 [============================>.] - ETA: 0s - loss: 0.1737 - acc: 0.9300

360000/360000 [==============================] - 919s 3ms/step - loss: 0.1737 - acc: 0.9300 - val_loss: 0.1677 - val_acc: 0.9324

Epoch 00002: val_acc improved from 0.92578 to 0.93240, saving model to weights.002-0.9324.hdf5
Epoch 3/10
  7712/360000 [..............................] - ETA: 14:32 - loss: 0.1641 - acc: 0.9307

...

360000/360000 [==============================] - 914s 3ms/step - loss: 0.1042 - acc: 0.9610 - val_loss: 0.1515 - val_acc: 0.9418

Epoch 00008: val_acc improved from 0.94168 to 0.94178, saving model to weights.008-0.9418.hdf5
Epoch 9/10
  7712/360000 [..............................] - ETA: 14:26 - loss: 0.0943 - acc: 0.9654

359968/360000 [============================>.] - ETA: 0s - loss: 0.0956 - acc: 0.9646

360000/360000 [==============================] - 913s 3ms/step - loss: 0.0956 - acc: 0.9646 - val_loss: 0.1532 - val_acc: 0.9417

Epoch 00009: val_acc did not improve from 0.94178
Epoch 10/10
 11392/360000 [..............................] - ETA: 14:11 - loss: 0.0828 - acc: 0.9711

359968/360000 [============================>.] - ETA: 0s - loss: 0.0868 - acc: 0.9682

360000/360000 [==============================] - 913s 3ms/step - loss: 0.0868 - acc: 0.9682 - val_loss: 0.1568 - val_acc: 0.9420

Epoch 00010: val_acc improved from 0.94178 to 0.94198, saving model to weights.010-0.9420.hdf5


In [0]:
sequencesTest = tokenizer.texts_to_sequences(data_test["text"].values)
dataTest = pad_sequences(sequencesTest, maxlen=988)
print(data.shape)
print(dataTest.shape)


(400000, 988)
(100000, 988)


In [0]:
print(dataTest.shape)


(100000, 988)


In [0]:
sequence_length = data.shape[1]
embedding_dim = 76
num_filters = 32
filter_sizes = [33, 33, 33]
drop = 0.2

inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

c0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

DP0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(c0)
DP1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(c1)
DP2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(c2)

CT = Concatenate(axis=1)([DP0, DP1, DP2])

flatten = Flatten()(CT)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax')(dropout)

batch_size=32
epochs = 10

model = Model(inputs=inputs, outputs=output)

In [0]:
model.load_weights("hello.hdf5")

In [0]:


yPred = model.predict(dataTest, verbose = 1)
answer = np.argmax(yPred,axis=1)



100000/100000 [==============================] - 79s 793us/step


In [0]:
newdataTest = pd.DataFrame({ 'label':answer})
newdataTest.index += 1
newdataTest.to_csv("Submission.csv")

In [0]:
newdataTest["label"].value_counts()

2    63749
0    20547
1    15704
Name: label, dtype: int64